In [6]:
search_for = 115
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.008170366287231445
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 228490141
type: [1, 1, 1, 1, 115] 115
cases of this type: 174900625
10000 17.778184138209898
20000 17.23280878968828
30000 17.608608778270906
40000 18.16896364521134
50000 80.51689720006176
60000 76.31252263199535
70000 86.38569746590731
80000 80.31409534354586
90000 89.4232103849253
100000 86.70170636734208
110000 87.5824863105623
120000 81.25522130621172
130000 88.55704291448504
140000 72.89065178851034
150000 100.50965865198496
160000 83.76729267246373
170000 101.17698513523094
180000 95.19598039632352
190000 107.44736970913785
200000 105.9497122023257
210000 126.77067425570498
220000 147.65862766912818
230000 166.29743450018816
240000 155.89047198896986
250000 153.9801629878491
260000 152.92775348721085
270000 155.69019952871008
280000 155.62935648405977
290000 156.76812487394315
300000 159.73340276220998
310000 158.21497151

3050000 41.97574553686585
3060000 41.89221841424445
3070000 40.3270130950276
3080000 40.31571328880157
3090000 39.95582506820996
3100000 41.131434396063746
3110000 66.61271316592347
3120000 208.96401105497495
3130000 152.70312551215588
3140000 166.49837510771007
3150000 137.0269841179651
3160000 167.02506232274794
3170000 157.0104250359974
3180000 166.9037069029505
3190000 161.80567196887182
3200000 173.66584878071802
3210000 159.26643024925696
3220000 171.91163857363065
3230000 156.56567534144116
3240000 169.43755548179007
3250000 160.12260808366943
3260000 163.0000286150177
3270000 161.68984346273206
3280000 156.7366796369309
3290000 159.4279362799199
3300000 152.19825123290826
3310000 160.67960219676635
3320000 149.37962695659826
3330000 170.03541858131936
3340000 157.59039664904768
3350000 161.78881600111373
3360000 162.77011249955552
3370000 164.66424075862597
3380000 165.61859916013864
3390000 154.3060943112563
3400000 168.52375042094684
3410000 171.07074477347226
3420000 177.579

6130000 40.08710756629708
6140000 52.66962794415637
6150000 43.34561935306007
6160000 46.937113452540636
6170000 39.54454037127973
6180000 103.78715507878239
6190000 191.41265158062853
6200000 177.2246909812598
6210000 152.83887030091535
6220000 167.56207073178376
6230000 158.0123094552804
6240000 171.13720403254885
6250000 156.59298857016003
6260000 173.12141349335292
6270000 156.16135720173983
6280000 170.77395540190383
6290000 149.83209686732383
6300000 169.80894187949798
6310000 153.5029486431927
6320000 166.81280915533927
6330000 156.7085729325001
6340000 165.37191451761944
6350000 165.34017389484325
6360000 160.79377897641197
6370000 160.005970706303
6380000 162.04393753053697
6390000 166.89193264899004
6400000 161.71540908461074
6410000 158.2219968014356
6420000 167.57669222922092
6430000 175.8562286413418
6440000 170.54523176894364
6450000 164.33547132891496
6460000 166.05594158402604
6470000 161.89461780469017
6480000 162.948684608556
6490000 154.62970036478922
6500000 156.266

9210000 39.84538095273417
9220000 38.833096906703766
9230000 39.02605719680807
9240000 38.41120724161902
9250000 134.7206823818544
9260000 172.32964799999388
9270000 176.5582350004738
9280000 155.40073179167607
9290000 160.98693349231348
9300000 151.99816986258858
9310000 163.01641783586098
9320000 163.94540082335965
9330000 164.30296357494467
9340000 151.39230880644456
9350000 160.79130477732213
9360000 155.52007328959493
9370000 148.7741241285785
9380000 147.49402833965323
9390000 147.98209085756025
9400000 151.0375824071088
9410000 149.44959371861754
9420000 156.20951815651694
9430000 157.90072923058125
9440000 148.31893811962283
9450000 156.29076396707978
9460000 168.24754118607552
9470000 164.40239681284282
9480000 161.48506209037316
9490000 165.7713490698358
9500000 155.88026988126524
9510000 159.8984315520082
9520000 156.18131910530383
9530000 164.69542467867208
9540000 153.79707139484725
9550000 151.17381475435354
9560000 152.9523769337018
9570000 164.12336041837435
9580000 154

12200000 159.81624641590722
12210000 146.81927605063078
12220000 147.52221836893804
12230000 165.64654555003654
12240000 178.8650370613997
12250000 138.16175824952452
12260000 38.35767029671971
12270000 36.58978978357431
12280000 36.612810753558044
12290000 39.41685056136971
12300000 38.64302234665388
12310000 38.00329456790839
12320000 145.54696968616557
12330000 188.3456252258924
12340000 159.98950797647322
12350000 149.55737539911163
12360000 142.79509936224966
12370000 163.33174608436556
12380000 153.03927714396573
12390000 161.74091214994866
12400000 151.17570262399164
12410000 160.90144666905618
12420000 149.70322276817035
12430000 150.8623383943232
12440000 148.1401944135228
12450000 157.21318358486297
12460000 152.95353416930723
12470000 149.77526654239534
12480000 151.26528061923364
12490000 157.57545219258844
12500000 157.3576082336945
12510000 145.02186942392453
12520000 153.53500877338016
12530000 151.12826476070373
12540000 151.82227014065307
12550000 148.26944107094212
12

...

214850000 2.8220813138003287
214860000 2.8465319967562093
214870000 3.1135074772498013
214880000 2.5553608726157875
214890000 2.4143040144622328
214900000 2.163904155081014
214910000 2.185020629495243
214920000 2.220361143084341
214930000 1.8475688412523072
214940000 3.1177685744781756
214950000 2.7194180996332236
214960000 2.4232920285682082
214970000 2.2997798366275295
214980000 2.2311818017070424
214990000 2.125788807589988
215000000 2.7548144109913775
215010000 2.2655429090185897
215020000 2.6528596799131634
215030000 2.743953653768599
215040000 2.903787866357518
215050000 1.6894879010490775
215060000 3.346555252907064
215070000 3.295794719921609
215080000 3.2860747637178296
215090000 3.152750171046363
215100000 2.218136318528672
215110000 3.276266643127878
215120000 2.7826144874598056
215130000 2.8560160836547017
215140000 2.155007357876539
215150000 2.1427942611562476
215160000 2.2677295130034754
215170000 2.741766104897082
215180000 2.457908255236546
215190000 2.545034166706767


217700000 0.6983509473209579
217710000 0.7238446676746011
217720000 0.7291091794066032
217730000 0.7196309004882905
217740000 0.4516014200527734
217750000 0.6394741146817804
217760000 0.6916337568901381
217770000 0.6008441935741172
217780000 0.565459479992032
217790000 0.390629167310132
217800000 0.3906473985389471
217810000 0.38762747787465657
217820000 0.3907884146491157
217830000 0.4043074330836468
217840000 0.5045796678185462
217850000 0.622936849987891
217860000 0.6210301819331844
217870000 0.52498683204633
217880000 0.46885927963300544
217890000 0.5481083305055101
217900000 0.5021038129808506
217910000 0.6818277157246669
217920000 0.6001899723707371
217930000 0.5758017364684939
217940000 0.6879636341816981
217950000 0.6143436142312486
217960000 0.4948562776477933
217970000 0.6773812578790188
217980000 0.5422004134783082
217990000 0.4063808155425986
218000000 0.38944264302062986
218010000 0.3884139019808438
218020000 0.4007354603678584
218030000 0.6752080844522185
218040000 0.6746

220520000 1.7211903675405582
220530000 1.187371225867669
220540000 1.6063679295435547
220550000 1.2447133083506887
220560000 1.6412058784205847
220570000 1.904626531230827
220580000 1.84178493206599
220590000 0.900777398397439
220600000 1.4422710717947085
220610000 1.9216621936636369
220620000 1.618870387503849
220630000 1.0978553659003971
220640000 1.6695909313393036
220650000 1.8163251188057792
220660000 1.4252632302381198
220670000 1.244750734345317
220680000 1.256625126896918
220690000 1.8296401232900221
220700000 1.8718241006456482
220710000 0.9015798399532239
220720000 1.354560108001113
220730000 1.2299519943345123
220740000 1.6357465620368585
220750000 1.53482682960097
220760000 1.4760797128803598
220770000 1.56241482736869
220780000 1.979364604287803
220790000 1.3595413666662575
220800000 1.6876667619039152
220810000 1.857563603406191
220820000 1.2620313313000666
220830000 1.113588534349084
220840000 1.7796589975762767
220850000 1.81745069220578
220860000 1.6977142030049033
220

223340000 0.9519985704880887
223350000 1.7835139484762352
223360000 1.3723789522162875
223370000 1.1897758403095338
223380000 1.308396458680537
223390000 1.6467950838464498
223400000 1.439815896275719
223410000 0.7979437068460583
223420000 1.1847163237638474
223430000 1.4496889268873399
223440000 1.5739524214684035
223450000 0.8717544530321162
223460000 1.2511171748041974
223470000 0.9438505926778382
223480000 1.3295982449998458
223490000 1.3875729719716907
223500000 1.0222729740343028
223510000 1.5083047404460908
223520000 1.750366569078909
223530000 1.3580203260058696
223540000 1.3995762964189054
223550000 1.5552186028858688
223560000 1.7881421847539147
223570000 1.2168909953325986
223580000 1.0940329233671493
223590000 1.1842720135210685
223600000 1.4966364422813654
223610000 0.8728104792429408
223620000 0.9552617314657172
223630000 0.8753399632100065
223640000 1.4437968811921942
223650000 1.025489367454383
223660000 1.0831915816993911
223670000 1.1107062096450064
223680000 1.380272

226150000 0.08653171853158871
226160000 0.08581182787744868
226170000 0.1430370722193387
226180000 0.12093892107105257
226190000 0.1456523793932398
226200000 0.11340389005894795
226210000 0.14422790007144212
226220000 0.14663588788642487
226230000 0.14448762932124404
226240000 0.10280184902153412
226250000 0.12752739707393118
226260000 0.1280405958736936
226270000 0.10877823022234441
226280000 0.10969302927715248
226290000 0.14333696259848278
226300000 0.1425009580256343
best so far: 0
type: [1, 1, 23, 1, 5] 115
cases of this type: 330625
226310000 0.1202045251522859
226320000 0.11966847038322687
226330000 0.13000643080880245
226340000 0.14981345616671773
226350000 0.14019719582449067
226360000 0.16303293663966656
226370000 0.15621973293436237
226380000 0.15490300744986532
226390000 0.15760131803780794
226400000 0.11084206146079964
226410000 0.08798274374830724
226420000 0.08743670377125343
226430000 0.08698602217376232
226440000 0.0867530734180808
226450000 0.08622990600870052
2264600